# Basic HW AES CPA attack.

**IMPORTANT NOTES!**
- **VCCIOB jumper (JP9)** must be in **top position (3.3V)**, which is different from the other test notebook;
- **nPOR jumper (JP5)** must be in **bottom position (1-3)**.

In [ ]:
import chipwhisperer as cw

In [ ]:
bsfile = '../fpga/vivado/cw341.runs/impl_20pin/cw341_top.bit'

In [ ]:
!ls -l $bsfile

In [ ]:
target = cw.target(None, cw.targets.CW340, bsfile=bsfile, force=True)

In [ ]:
target.fpga_buildtime

In [ ]:
target.pll.pll_enable_set(True)
target.pll.pll_outenable_set(False, 0)
target.pll.pll_outenable_set(True, 1)
target.pll.pll_outenable_set(True, 2)

target.pll.pll_outfreq_set(10E6, 1)
target.pll.pll_outfreq_set(10E6, 2)

In [ ]:
scope = cw.scope()
scope.default_setup()

**Once the `husky_clock_fix` branch is merged** (i.e. issues [490](https://github.com/newaetech/chipwhisperer/issues/490), [499](https://github.com/newaetech/chipwhisperer/issues/501) and [501](https://github.com/newaetech/chipwhisperer/issues/501) are closed), 4x sampling works well, and `CLOCKS_FIXED` can be set to `True`.

Otherwise, we crank up to 20x sampling (`CLOCKS_FIXED = False`) to better catch the leakage.

In [ ]:
CLOCKS_FIXED = False

if CLOCKS_FIXED:
    scope.clock.adc_mul = 4
    scope.clock.clkgen_freq = 10e6
    scope.clock.clkgen_src = 'extclk'
    scope.clock.adc_phase = 15
    scope.adc.samples = 60
else:
    scope.clock.adc_mul = 20
    scope.clock.clkgen_freq = 10e6
    scope.clock.clkgen_src = 'extclk'
    scope.clock.adc_phase = 0
    scope.adc.samples = 300

scope.adc.presamples = 0
scope.gain.db = 32

assert scope.clock.adc_locked

In [ ]:
project_file = "projects/Tutorial_HW_CW340.cwp"
project = cw.create_project(project_file, overwrite=True)

In [ ]:
from tqdm.notebook import tnrange
import numpy as np
import time
from Crypto.Cipher import AES

ktp = cw.ktp.Basic()

traces = []
textin = []
keys = []
N = 10000  # Number of traces

# initialize cipher to verify DUT result:
key, text = ktp.next()
cipher = AES.new(bytes(key), AES.MODE_ECB)

for i in tnrange(N, desc='Capturing traces'):
    # run aux stuff that should come before trace here

    key, text = ktp.next()  # manual creation of a key, text pair can be substituted here
    textin.append(text)
    keys.append(key)
    
    ret = cw.capture_trace(scope, target, text, key)
    if not ret:
        print("Failed capture")
        continue

    assert (list(ret.textout) == list(cipher.encrypt(bytes(text)))), "Incorrect encryption result!\nGot {}\nExp {}\n".format(ret.textout, list(text))
    #trace += scope.getLastTrace()
        
    traces.append(ret.wave)
    project.traces.append(ret)

In [ ]:
numtraces = 100

from bokeh.palettes import inferno
from bokeh.plotting import figure, show
from bokeh.resources import INLINE
from bokeh.io import output_notebook
from bokeh.models import Span, Legend, LegendItem
import itertools
output_notebook(INLINE)

B = figure(plot_width=1800)
colors = itertools.cycle(inferno(numtraces))
for i in range(numtraces):
    B.line(xrange, traces[i], color=next(colors))
show(B)


In [ ]:
project.save()

In [ ]:
import chipwhisperer as cw
import chipwhisperer.analyzer as cwa
project_file = "projects/Tutorial_HW_CW340"
project = cw.open_project(project_file)
attack = cwa.cpa(project, cwa.leakage_models.last_round_state_diff)
cb = cwa.get_jupyter_callback(attack)

In [ ]:
attack_results = attack.run(cb)

In [ ]:
avg_pge = np.average(attack_results.pge)
if CLOCKS_FIXED:
    MAX_PGE = 10
else:
    MAX_PGE = 50
assert avg_pge < MAX_PGE, 'Average PGE is %d' % avg_pge